# Interface utilisateur pour Sakila

## 1. Créer un fonction connect_db() qui prendra en entrée un dictionnaire contenant un nom d’utilisateur, un mot de passe, un host et le nom d’une base de données

In [1]:
# https://docs.sqlalchemy.org/en/14/core/engines.html

from sqlalchemy import create_engine
import pandas as pd


#with open("nomduficher") as fichier:
#   dico = Json.load(fichier["mysql"])

In [15]:
def connect_db(dico):

    str_engine = "mysql+" + dico["connector"] + "://" + dico["user"] + ":" + dico["pwd"] + "@" + dico["host"] + ":" + dico["port"] + "/" + dico["bdd"]
    connect = create_engine(str_engine)
    return connect

d = {
    "connector": "pymysql",
    "user" : "simplon",
    "pwd" : "fidelio",
    "host" : "localhost",
    "port": "3306",
    "bdd" : "sakila"
}

engine = connect_db(d)

## 2. Créer une fonction new_customer() qui va permettre l’insertion d’un nouveau client dans la table customer en demandant à un utilisateur de saisir les données, ainsi que son adresse pour remplir les tables address, city et country. Faire attention aux contraintes d’intégrités

In [6]:
def new_customer(engine):

    country = input("Pays: ")
    country_df = pd.read_sql_query("select country_id from country where country = '%s';" %(country), engine)
    if country_df.empty:
        engine.execute('insert into country (country) values ("%s");' %(country))
        country_df = pd.read_sql_query("select country_id from country where country = '%s';" %(country), engine)

    city = input("Ville: ")
    city_df = pd.read_sql_query("select city_id from city where city = '%s';" %(city), engine)
    if city_df.empty:
        engine.execute('insert into city (city, country_id) values ("%s",%s);' %(city, int(country_df["country_id"][0])))
        city_df = pd.read_sql_query("select city_id from city where city = '%s';" %(city), engine)

    address = input("Adresse: ")
    address2 = input("Adresse 2: ")
    district = input("District: ")
    postal_code = input("Code Postal: ")
    phone = input("Numéro de téléphone: ")
    location = "ST_GeomFromWKB(X'0101000000000000000000F03F000000000000F03F')"
    address_df = pd.read_sql_query("select address_id from address where address = '%s';" %(address), engine)
    if address_df.empty:
        engine.execute('insert into address (address, address2, district, city_id, postal_code, phone) values ("%s","%s","%s",%s,"%s","%s",%s);' %(address, address2, district, int(city_df["city_id"][0]), postal_code, phone, location))
    address_df = pd.read_sql_query("select address_id from address where address = '%s';" %(address), engine)

    store = int(input("Store (1 ou 2): "))
    first_name = input("Prénom: ")
    last_name = input("Nom: ")
    email = input("Email: ")
    customer_df = pd.read_sql_query("select customer_id from customer where email= '%s';" %(email), engine)
    if customer_df.empty:
        engine.execute('insert into customer (store_id, first_name, last_name, email, address_id) values (%s,"%s","%s","%s",%s);' %(store, first_name, last_name, email, int(address_df["address_id"][0])))
        customer_df = pd.read_sql_query("select customer_id from customer where email= '%s';" %(email), engine)

new_customer(engine)

##  3. Ecrire une fonction qui va afficher l’historique de location d’un client

In [13]:
def historique(engine):

    historique_customer = pd.read_sql_query("select * from rental where customer_id = (select customer_id from customer where customer_id=599);",engine)
    return historique_customer

historique(engine)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1008,2005-05-31 01:18:56,914,599,2005-06-01 01:24:56,2,2006-02-15 21:30:53
1,2272,2005-06-18 06:29:53,1364,599,2005-06-23 10:58:53,1,2006-02-15 21:30:53
2,3043,2005-06-20 12:38:35,633,599,2005-06-29 14:16:35,2,2006-02-15 21:30:53
3,3398,2005-06-21 15:34:38,221,599,2005-06-29 11:23:38,1,2006-02-15 21:30:53
4,3429,2005-06-21 18:46:05,456,599,2005-06-30 17:28:05,1,2006-02-15 21:30:53
5,5065,2005-07-09 04:42:00,1788,599,2005-07-12 08:55:00,1,2006-02-15 21:30:53
6,5843,2005-07-10 17:14:27,1615,599,2005-07-15 21:18:27,2,2006-02-15 21:30:53
7,6800,2005-07-12 17:03:56,3742,599,2005-07-21 20:32:56,2,2006-02-15 21:30:53
8,6895,2005-07-12 21:23:59,2935,599,2005-07-19 20:47:59,2,2006-02-15 21:30:53
9,8965,2005-07-30 03:52:37,2419,599,2005-08-05 01:28:37,2,2006-02-15 21:30:53


## 4. D’après vous quels seraient les indicateurs graphiques les plus pertinents pour suivre l’évolution du business de location de vidéos? Avec Matplotlib et Seaborn rédiger trois fonctions qui vont vous permettre d’afficher les trois graphiques permettant de suivre au mieux la rentabilité de l’affaire

In [24]:
customer1 = pd.read_sql_query("select rental.rental_date, amount, customer.first_name, customer.last_name from payment inner join customer on payment.customer_id = customer.customer_id inner join rental on customer.customer_id = rental.customer_id;", engine)
customer1

,rental_date,amount,first_name,last_name
0,2005-05-25 11:30:37,2.99,MARY,SMITH
1,2005-05-25 11:30:37,0.99,MARY,SMITH
2,2005-05-25 11:30:37,5.99,MARY,SMITH
3,2005-05-25 11:30:37,0.99,MARY,SMITH
4,2005-05-25 11:30:37,9.99,MARY,SMITH
...,...,...,...,...
445478,2005-08-23 11:25:00,4.99,AUSTIN,CINTRON
445479,2005-08-23 11:25:00,1.99,AUSTIN,CINTRON
445480,2005-08-23 11:25:00,8.99,AUSTIN,CINTRON
445481,2005-08-23 11:25:00,2.99,AUSTIN,CINTRON


In [25]:
import matplotlib as mtp
import matplotlib.pyplot as plt
import seaborn as sns

plt.subplots(figsize = (10,5), dpi=70)
plt.title("Client les plus rentable")

client_plus_rentable = pd.read_sql_query(customer1,engine)
client_plus_rentable

sns.plot(x="rental_date",y="amount",data=customer1)
plt.xticks(rotation=90)
plt.show()
plt.close()

ObjectNotExecutableError: Not an executable object:                rental_date  amount first_name last_name
0      2005-05-25 11:30:37    2.99       MARY     SMITH
1      2005-05-25 11:30:37    0.99       MARY     SMITH
2      2005-05-25 11:30:37    5.99       MARY     SMITH
3      2005-05-25 11:30:37    0.99       MARY     SMITH
4      2005-05-25 11:30:37    9.99       MARY     SMITH
...                    ...     ...        ...       ...
445478 2005-08-23 11:25:00    4.99     AUSTIN   CINTRON
445479 2005-08-23 11:25:00    1.99     AUSTIN   CINTRON
445480 2005-08-23 11:25:00    8.99     AUSTIN   CINTRON
445481 2005-08-23 11:25:00    2.99     AUSTIN   CINTRON
445482 2005-08-23 11:25:00    2.99     AUSTIN   CINTRON

[445483 rows x 4 columns]